# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load csv file 
city_df = pd.read_csv("../WeatherPy/output_results/city_rm.csv")
# show data
city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Muisne,75,EC,1593898277,78,0.60,-80.03,82.40,8.05
1,Lagoa,74,PT,1593898277,87,39.05,-27.98,69.44,5.41
2,Mayo,90,US,1593898277,62,38.89,-76.51,87.80,5.82
3,Hilo,40,US,1593898277,62,19.73,-155.09,84.20,9.17
4,Bilibino,100,RU,1593898278,78,68.05,166.44,49.60,4.50
...,...,...,...,...,...,...,...,...,...
557,San Fernando del Valle de Catamarca,0,AR,1593898346,54,-28.47,-65.79,64.00,1.01
558,Elmina,100,GH,1593898346,89,5.09,-1.35,75.25,8.81
559,Flinders,0,AU,1593898346,68,-34.58,150.86,52.00,3.00
560,Atar,0,MR,1593898346,15,20.52,-13.05,99.25,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# assign latitude and longitude
Lat_Lng = city_df[["Lat", "Lng"]]

# assign humidity 
hum = city_df["Humidity"]

In [4]:
# Heatmap of humidity
location = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=300, point_radius=7)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#define cities to search
p_city_df = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) & \
                                   (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 80)].dropna()

p_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
157,Kamenka,0,RU,1593898296,47,51.32,42.77,72.00,6.31
215,Suez,0,EG,1593898247,70,29.97,32.53,76.33,6.98
358,Akşehir,0,TR,1593898321,34,38.36,31.42,71.69,3.58
359,Oleksandrivka,0,UA,1593898321,43,46.84,32.76,77.74,7.72
364,Qui Nhon,0,VN,1593898120,76,13.77,109.23,77.85,5.64
383,Mbandaka,0,CD,1593898324,79,0.05,18.26,72.64,2.57
389,Benghazi,0,LY,1593898325,78,32.12,20.07,77.45,7.47
406,Marsá Maţrūḩ,0,EG,1593898327,94,31.35,27.25,75.20,5.82
434,Kunduz,0,AF,1593898331,19,36.73,68.86,73.49,2.82
446,Gorgan,0,IR,1593898332,94,36.84,54.43,71.60,4.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a field for hotel names
hotel_df = p_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# include column for hotel names
hotel_df["Hotel Name"] = ""

# Display
hotel_df

,City,Country,Lat,Lng,Hotel Name
157,Kamenka,RU,51.32,42.77,
215,Suez,EG,29.97,32.53,
358,Akşehir,TR,38.36,31.42,
359,Oleksandrivka,UA,46.84,32.76,
364,Qui Nhon,VN,13.77,109.23,
383,Mbandaka,CD,0.05,18.26,
389,Benghazi,LY,32.12,20.07,
406,Marsá Maţrūḩ,EG,31.35,27.25,
434,Kunduz,AF,36.73,68.86,
446,Gorgan,IR,36.84,54.43,


In [7]:
map_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
#find info for each row
for index, row in hotel_df.iterrows():
    # pull in needed data
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # use the keyword for params
    params["location"] = f"{lat},{lng}"

    # API request 
    print(f"display the results {index}: {city_name}.")
    response = requests.get(map_url, params=params).json()
    
    #results
    final_results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Nearby hotel in {city_name} : {final_results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = final_results[0]['name']

    # No hotel close to the city
    except (KeyError, IndexError):
        print("N/A")
        

display the results 157: Kamenka.
N/A
display the results 215: Suez.
Nearby hotel in Suez : LUSINDA HOTEL.
display the results 358: Akşehir.
Nearby hotel in Akşehir : Garp Hotel.
display the results 359: Oleksandrivka.
N/A
display the results 364: Qui Nhon.
Nearby hotel in Qui Nhon : Galaxy Hotel.
display the results 383: Mbandaka.
Nearby hotel in Mbandaka : Nina River Hotel.
display the results 389: Benghazi.
Nearby hotel in Benghazi : Ishbiliyah Beach Hotel.
display the results 406: Marsá Maţrūḩ.
Nearby hotel in Marsá Maţrūḩ : Beau Site.
display the results 434: Kunduz.
Nearby hotel in Kunduz : عسکرشاه.
display the results 446: Gorgan.
Nearby hotel in Gorgan : Hotel Saray Khan.
display the results 449: Sarakhs.
Nearby hotel in Sarakhs : Doosti Hotel.
display the results 545: Kiryat Gat.
Nearby hotel in Kiryat Gat : Desert Gat.


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display
fig

Figure(layout=FigureLayout(height='420px'))